In [1]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler 
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold  
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler 
import lightgbm as lgbm
from catboost import CatBoostClassifier 
from xgboost import XGBClassifier

# import pycaret library for binary classification 
from pycaret.classification import *

In [2]:
#setup train and test dataframes 
train = pd.read_csv('train.csv') 
test = pd.read_csv('test_x.csv') 
submission = pd.read_csv('sample_submission.csv') 

train = train.drop(columns=['index']) 
test = test.drop(columns=['index'])

In [3]:
clf = setup(data = train, target = 'voted')

Setup Succesfully Completed!


,Description,Value
0,session_id,5965
1,Target Type,Binary
2,Label Encoded,"1: 0, 2: 1"
3,Original Data,"(45532, 77)"
4,Missing Values,False
5,Numeric Features,41
6,Categorical Features,35
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [4]:
best_7 = compare_models(sort = 'AUC', n_select = 7)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
0,CatBoost Classifier,0.6944,0.7649,0.6583,0.7521,0.7020,0.3913,0.3950,9.9865
1,Gradient Boosting Classifier,0.6944,0.7648,0.6374,0.7645,0.6952,0.3938,0.4004,18.3120
2,Light Gradient Boosting Machine,0.6947,0.7642,0.6424,0.7621,0.6971,0.3939,0.3998,0.7598
3,Linear Discriminant Analysis,0.6913,0.7618,0.6604,0.7460,0.7005,0.3844,0.3875,0.7163
4,Extra Trees Classifier,0.6923,0.7601,0.6455,0.7562,0.6964,0.3884,0.3935,1.9032
5,Ada Boost Classifier,0.6899,0.7573,0.6543,0.7472,0.6976,0.3822,0.3858,4.2660
6,Extreme Gradient Boosting,0.6743,0.7445,0.6592,0.7213,0.6888,0.3487,0.3503,23.9597
7,Random Forest Classifier,0.6590,0.7155,0.6106,0.7229,0.6619,0.3229,0.3276,0.2533
8,Decision Tree Classifier,0.6139,0.6105,0.6467,0.6470,0.6469,0.2211,0.2211,1.4099
9,K Neighbors Classifier,0.5174,0.5148,0.5541,0.5593,0.5567,0.0273,0.0273,0.5275


Now, let us tune some models in our best_7 list and use them for prediction. 

In [6]:
lgbm = best_7[2] 

In [8]:
tuned_lgbm = tune_model(lgbm, n_iter = 100, optimize = 'AUC')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7033,0.7808,0.6466,0.7735,0.7044,0.4113,0.4179
1,0.6926,0.7624,0.6254,0.7692,0.6899,0.3915,0.3998
2,0.6912,0.7623,0.6397,0.7580,0.6938,0.3869,0.3926
3,0.7025,0.7730,0.6426,0.7751,0.7026,0.4102,0.4175
4,0.6975,0.7715,0.6477,0.7634,0.7008,0.3991,0.4046
5,0.6938,0.7616,0.6213,0.7741,0.6894,0.3944,0.4037
6,0.6894,0.7590,0.6202,0.7672,0.6859,0.3854,0.3940
7,0.6894,0.7583,0.6282,0.7620,0.6887,0.3844,0.3916
8,0.6991,0.7665,0.6240,0.7815,0.6939,0.4052,0.4152
9,0.6900,0.7609,0.6315,0.7607,0.6901,0.3853,0.3920


In [17]:
catboost = best_7[0] 

In [18]:
tuned_catboost = tune_model(catboost, n_iter = 100, optimize = 'AUC')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7067,0.7826,0.6655,0.7672,0.7127,0.4162,0.4206
1,0.6960,0.7660,0.6380,0.7669,0.6965,0.3972,0.4040
2,0.6944,0.7654,0.6558,0.7535,0.7012,0.3915,0.3955
3,0.7057,0.7770,0.6604,0.7689,0.7105,0.4147,0.4197
4,0.6950,0.7740,0.6552,0.7548,0.7015,0.3929,0.3970
5,0.6953,0.7652,0.6345,0.7681,0.6949,0.3961,0.4033
6,0.6894,0.7635,0.6328,0.7591,0.6902,0.3838,0.3902
7,0.6881,0.7602,0.6351,0.7556,0.6901,0.3809,0.3867
8,0.7022,0.7676,0.6366,0.7782,0.7003,0.4103,0.4185
9,0.6906,0.7644,0.6401,0.7564,0.6934,0.3855,0.3910


In [31]:
best_tuned = [] 
best_tuned.append(tuned_catboost) 
best_tuned.append(tuned_lgbm)
best_tuned.append(best_7[1]) 
best_tuned.append(best_7[3]) 
best_tuned.append(best_7[4]) 
best_tuned.append(best_7[5]) 
best_tuned.append(best_7[6]) 


In [32]:
blended = blend_models(estimator_list = best_tuned, fold = 5, method = 'soft')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.6991,0.7714,0.6517,0.7634,0.7032,0.4020,0.4072
1,0.6982,0.7705,0.6606,0.7566,0.7054,0.3989,0.4028
2,0.6933,0.7669,0.6466,0.7571,0.6975,0.3903,0.3954
3,0.6883,0.7603,0.6367,0.7548,0.6907,0.3810,0.3866
4,0.6930,0.7644,0.6422,0.7592,0.6958,0.3902,0.3958
Mean,0.6944,0.7667,0.6476,0.7582,0.6985,0.3925,0.3975
SD,0.0039,0.0041,0.0082,0.0030,0.0052,0.0074,0.0070


In [33]:
pred_holdout = predict_model(blended)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.6971,0.7654,0.647,0.763,0.7002,0.3982,0.4038


In [34]:
final_model = finalize_model(blended)


In [35]:
predictions = predict_model(final_model, data = test)


In [36]:
submission['voted'] = predictions['Score']


In [37]:
submission.head()

,index,voted
0,0,0.6580
1,1,0.8278
2,2,0.4684
3,3,0.2332
4,4,0.7263


In [38]:
submission.to_csv('partial_tuned.csv',index=False)

In [39]:
# take average  
soft_vote = pd.read_csv('autoML_probability_7.csv') 
blend = pd.read_csv('stack_lgbm.csv')
avg_score = (submission['voted'] + soft_vote['voted'] + blend['voted'])/3.0

In [40]:
submission['voted'] = avg_score 
submission.head()

,index,voted
0,0,0.624700
1,1,0.853267
2,2,0.476300
3,3,0.218000
4,4,0.729233


In [41]:
submission.to_csv('additional_blend.csv',index=False)

We will do stacking using xgboost